In [1]:
import matplotlib.pyplot as plt
from nilearn.glm.first_level import FirstLevelModel
from nilearn import image, plotting
import pandas as pd
from __future__ import division
import shutil as sh
from os.path import join 
from glob import glob
import itertools
from distutils.dir_util import copy_tree
import shutil as shutil
from glob import glob
import pandas as pd
import numpy as np
from subprocess import check_call
import matplotlib.pyplot as plt
#from utilities import ensure_dir, get_n_vols
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve
from nipype.interfaces.dcm2nii import Dcm2niix, Dcm2nii
#import nipype_init_new
#nipype_init_new.init()
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix,ConnectivityMeasure
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_matrix, plot_glass_brain
from nilearn.image import concat_imgs, mean_img, load_img, index_img, math_img, resample_img
from nilearn.masking import apply_mask
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_design_matrix
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn import plotting
from statsmodels.sandbox.stats.multicomp import multipletests as multi
from scipy.stats import norm
import statsmodels.stats.multitest as ssm
import statsmodels
import seaborn as sns
from scipy.stats import pearsonr,ttest_ind
from sklearn.model_selection import StratifiedKFold,permutation_test_score
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
import time

/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def pad_vector(contrast_, n_columns):
    """A small routine to append zeros in contrast vectors"""
    return np.hstack((contrast_, np.zeros(n_columns - len(contrast_))))
def ensure_dir(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)
def get_n_vols(iname):
    img = load_img(iname)
    dims=np.shape(image.get_data(img))
    return 1 if len(dims) < 4 else dims[3]
def compute_contrast(design_matrices,contrast_name):
    dmcols=[design_matrices[0].columns.values.tolist(),design_matrices[1].columns.values.tolist(),
            design_matrices[2].columns.values.tolist(),design_matrices[3].columns.values.tolist(

                
            )]
    for i in range(len(dmcols)):
        for j in range(len(dmcols[i])):
            if contrast_name==dmcols[i][j]:
                dmcols[i][j]=1
            else:
                dmcols[i][j]=0
    return dmcols
def list_substract(a,b):
    new_list=[]
    for i in range(len(a)):
        new_list.append([])
        for j in range(len(a[i])):
            new_list[i].append(0)
            new_list[i][j]=a[i][j]-b[i][j]
    return new_list

In [4]:
import os
base_dir='/home/haiyanwu/nas_data/experiment_data/uarep_only/'
beta_dir=base_dir+'1.GLM/results/GLM6uarep_combination/betamap/'
ensure_dir(beta_dir)

In [5]:
subList=[101,102,103,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,123,125,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143]

In [6]:
data=pd.read_csv('/home/haiyanwu/nas_data/experiment_data/uarep_only/2.uarep-hddm/fMRI_sessionData_DDM_rep.csv')
data['time_onset_new']=data['time_onset']+2
data['condition_run']=list(map(lambda x,y: '%s_%d'%(x,y),data['condition'],data['round']))
data['duration']=data['RT']
data

,Unnamed: 0,subj,trial,round,menu,menu_position_1,menu_position_2,true_answer,condition,reinforcer,...,session,money_diff,former_lie,former_honest,consis_diff,diff,former_diff,time_onset_new,condition_run,duration
0,0,101,1,1,1,1,37,1,enhance_honesty,4,...,1,100,0,0,0,-4,0,2.000000,enhance_honesty_1,1.551243
1,1,101,2,1,25,25,61,25,enhance_honesty,2,...,1,100,0,0,0,-2,0,8.821815,enhance_honesty_1,1.775196
2,2,101,3,1,6,6,42,6,enhance_random,6,...,1,100,0,0,0,-6,0,15.262858,enhance_random_1,1.734819
3,3,101,4,1,2,38,2,2,enhance_lie,8,...,1,100,0,0,0,8,0,23.975525,enhance_lie_1,2.359468
4,4,101,5,1,9,45,9,9,enhance_random,6,...,1,100,0,0,0,-6,0,30.468093,enhance_random_1,1.625446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10651,283,143,68,8,32,32,68,32,enhance_lie,2,...,4,100,0,7,-7,2,-7,455.473537,enhance_lie_8,1.413876
10652,284,143,69,8,27,63,27,27,enhance_random,8,...,4,100,1,6,-5,-8,-5,462.984077,enhance_random_8,0.796888
10653,285,143,70,8,30,66,30,30,enhance_random,6,...,4,100,0,7,-7,-6,-7,469.605028,enhance_random_8,1.154773
10654,286,143,71,8,11,47,11,11,enhance_lie,2,...,4,100,0,7,-7,2,-7,477.095536,enhance_lie_8,1.238054


In [6]:
# considering some over time trials were removed so the trial number of each session was not the same, we didn't combin four sessions together to run the GLM 

In [10]:

all=data

tr=1

for subno in subList:

    subdir_beta=beta_dir+"sub-%03d/"%subno
    ensure_dir(subdir_beta)

    
  
    d=all[all['subj']==subno]
    #d=d[d['RT']<=4]
    d.reset_index(inplace=True)
    d['combination']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),d['former_diff'],d['diff'],d['islie']))
    cnames=list(set((d['combination'].to_list())))





    n_columns=[]
    design_matrices=[]
    func_data=[]
    for sesno in [1,2,3,4]:
        ## Prepare data and analysis parameters
        # load functional images
        fd=load_img(base_dir+"fmriprep-deriv-rep/fmriprep/sub-%03d/ses-%01d/func/sub-%03d_ses-%01d_preproc_only_masked.nii.gz"%(subno,sesno,subno,sesno))
        n_scans=get_n_vols(fd)
        func_data.append(fd)

        # load the events file
        events=d.loc[d.session==sesno,:]
        events=events.rename(columns={'time_onset_new':'onset'})
     
        events=events.reset_index(drop=True)
        events['trial_type']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),events['former_diff'],events['diff'],events['islie']))
        #events.loc[events['RT']>4,'trial_type']='exclusion'
        events=events[['onset','trial_type','duration']]
        events=events.reset_index(drop=True)

        # load the confound file
        confounds=pd.read_table(base_dir+"1.GLM/confound_files_rep/sub-%03d/ses-%01d/sub-%03d_ses-%01d_confound.txt"%(subno,sesno,subno,sesno),header=None,sep=',')

        frame_times=np.arange(n_scans) * tr
        dm=make_first_level_design_matrix(
                    frame_times=frame_times, events=events, drift_model='cosine', high_pass=1/128,
                    add_regs=confounds, hrf_model='spm',min_onset=0)
        n_columns.append(dm.shape[1])
        design_matrices.append(dm)



   
    glm_comb = FirstLevelModel(t_r=tr,
                        slice_time_ref=0.5, # default is 0, but in SPM default is 1; used the middle slice
                        hrf_model='spm', # block design: spm; event-related: spm + derivative
                        noise_model='ar1',
                        drift_model='cosine',
                        high_pass=1/128,
                        standardize=False,
                        min_onset=0,
                        mask_img=False,
                        smoothing_fwhm=None,
                        verbose=0
                        )
    starttime = time.time()
    glm_comb = glm_comb.fit(func_data,design_matrices=design_matrices)
    print("- %f"%(time.time()-starttime))



    basic_contrasts=dict()
    # Generate the contrasts
    for cname in cnames:
        temp_name=cname
        temp_contrast=compute_contrast(design_matrices,cname)
        basic_contrasts[temp_name]=temp_contrast




    from nilearn import plotting
    # Iterate on contrasts
    for index, (contrast_id, contrast_val) in enumerate(basic_contrasts.items()):
        
        ## estimate the contasts
        ## note that the model implictly compute a fixed effects across the two sessions
        ## effect size
        eff_map = glm_comb.compute_contrast(
            contrast_val, output_type='effect_size')
       

        eff_map.to_filename(join(subdir_beta, 'sub-%03d_%s_eff_map.nii.gz'%(subno,contrast_id)))
        

- 272.762374
- 242.138324
- 240.525078
- 233.498989
- 255.157413
- 241.583659
- 245.940547
- 240.976053
- 227.586100
- 253.341546
- 252.420439
- 244.869015
- 256.080314
- 286.232093
- 293.391490
- 316.097299
- 319.493903
- 311.492645
- 327.105882
- 292.205286
- 311.746905
- 298.709989
- 251.805140
- 322.164086
- 340.218024
- 298.211097
- 293.393084
- 315.554316
- 367.820280
- 269.407282
- 297.506967
- 290.091878
- 316.558259
- 284.419942
- 326.507280
- 405.436145
- 308.563497
